# Install & Import libraries

In [ ]:
import numpy as np
from pythainlp.tokenize import word_tokenize, Tokenizer
from pythainlp.corpus.common import thai_words
from pythainlp.corpus import thai_stopwords
import pandas as pd
from difflib import SequenceMatcher, Differ, get_close_matches
from pythainlp.tokenize import sent_tokenize
from pythainlp.spell import correct
import deepcut
import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.utils import plot_model
import matplotlib.pyplot as plt
from IPython.display import clear_output
import plotly
import plotly.graph_objs as go
import pickle as p
from gensim.models import Word2Vec

# Import CSV file

In [ ]:
file_path='/content/drive/MyDrive/K-DAI/Y2S1/Text mining/Week4/Assignmente'
df=pd.read_csv(file_path,encoding ='utf-8' ,  usecols=['reply_no','reply_text'], header=0)

In [ ]:
df.head()

,reply_no,reply_text
0,1,กลับมาเปิดอีกรอบหลังจากหายไปนานหลายปีครับ ฟังค...
1,2,ปรับปรุง 2013-02-08- เปลี่ยนวิธีสุ่ม ID ของโพส...
2,3,ดีใจมากครับ ขอให้เปิดนานๆ
3,4,ID มีคำว่า god ต่อท้ายด้วย สุดยอด
4,5,"ขอเทสต์มั่งจิ๊ ""ทำไมโพสต์รูปไม่ได้"""


เพราะว่าข้อมูลมีจำนวน 60,000 row จึงต้องการ Sample มาแค่ 200 row เพื่อใช้ในการศึกษา

In [ ]:
df_samp=df.sample(n=200,random_state=1234)
df_samp.head(50)

,reply_no,reply_text
6268606,402,พอโดนตบเกรียนแตกเมื่อคืนเลยอาละวาดใหญ่ โง่ระดั...
748309,197,ใครมีดอค TOT บ้างวะแบ่งหน่อย ปิดรับไปยัง?
3581181,756,>>753 >>754 เรื่องหลิงซี ตอนเช้ากูตัดใจสั่งใน...
7339110,961,นนนนาาาาาาาาาาาาาาาาาาาาาาาาาาาาาาาาาาาาาาาาาา...
7282645,930,>>929 มึงมันโม่งปลอมตัวเป็นแม่ลม แม่ลมตัวจริงจ...
3004996,74,คลิปประกาศเมมเบอร์ตำแหน่งเซ็นเตอร์ซิงเกิ้ลที่ ...
662964,341,เรื่องร่างสุดยอดเทลม่อนนี่ติ่งแม่งเถียงกันฉิบห...
938820,640,>>636 เพราะบซซ.เป็นนางฟ้าฮะเลยมีสาวกตามเลียเยอ...
6962649,291,>>289 worldwide
1226537,216,>>145 ลองลงดู ตอนนี้มาม่าแม่งเริ่มเบาละ คนอื่น...


In [ ]:
df_samp.replace(r'[^ก-๙0-9]','',regex=True, inplace = True)
#df_samp.replace(r'\d+(.*?)[\u263a-\U0001f645]','',regex=True, inplace = True) ## For emoji excluding
df_samp.head(50)

,reply_no,reply_text
6268606,402,พอโดนตบเกรียนแตกเมื่อคืนเลยอาละวาดใหญ่โง่ระดับ...
748309,197,ใครมีดอคบ้างวะแบ่งหน่อยปิดรับไปยัง
3581181,756,753754เรื่องหลิงซีตอนเช้ากูตัดใจสั่งในชอปปี้ไป...
7339110,961,นนนนาาาาาาาาาาาาาาาาาาาาาาาาาาาาาาาาาาาาาาาาาา...
7282645,930,929มึงมันโม่งปลอมตัวเป็นแม่ลมแม่ลมตัวจริงจะรับ...
3004996,74,คลิปประกาศเมมเบอร์ตำแหน่งเซ็นเตอร์ซิงเกิ้ลที่4...
662964,341,เรื่องร่างสุดยอดเทลม่อนนี่ติ่งแม่งเถียงกันฉิบห...
938820,640,636เพราะบซซเป็นนางฟ้าฮะเลยมีสาวกตามเลียเยอะต่า...
6962649,291,289
1226537,216,145ลองลงดูตอนนี้มาม่าแม่งเริ่มเบาละคนอื่นลงตาม...


#Import Custom word, Correct word and Stop word list

In [ ]:
from pythainlp.corpus.common import thai_words
from pythainlp.corpus import thai_stopwords

In [ ]:
CUSTOM_WORDS ='/content/drive/MyDrive/K-DAI/Y2S1/Text mining/Week4/Custom_word.txt'
CORRECTING_WORDS='/content/drive/MyDrive/K-DAI/Y2S1/Text mining/Week4/Correcting_word.txt'
STOP_WORDS='/content/drive/MyDrive/K-DAI/Y2S1/Text mining/Week4/Stop_word.txt'


In [ ]:
copy_df=df_samp.copy()

#Define function for solve repeating alphabet and number

In [ ]:
import re

def transform_string(input_string):
    # Define a regular expression pattern to match repeating characters
    pattern = r'(.)\1{3,}'

    # Function to handle the replacement for each match found
    def replace(match):
        char = match.group(1)
        if char == '5' or char == 'ถ':
            return char * 3
        else:
            return char

    # Apply the transformation using the regular expression
    modified_string = re.sub(pattern, replace, input_string)
    return modified_string

#Example
text='สวัสดีครับบบ55555555ถถถถถถ'
text_clean=transform_string(text)
print('Text berfore clean: ',text)
print('Text after clean: ',text_clean)


Text berfore clean:  สวัสดีครับบบ55555555ถถถถถถ
Text after clean:  สวัสดีครับบบ555ถถถ


function for solve repeating alphabet, number and vowel

In [ ]:
##มี error ตรงคำซ้ำในประโยคเช่น แปป ผลที่ออกมาเป็น แป
"""
def modify_string(s):
    if not isinstance(s, str):
        return s  # return the original value if it is not a string

    vowels = 'ะัาำิีึ'
    max_repeats = {'5': 3, 'ถ': 3}
    result = ''

    i = 0
    while i < len(s):
        char = s[i]
        max_repeat = max_repeats.get(char, 1 if char in vowels else 1)

        count = 0
        while i < len(s) and s[i] == char:
            count += 1
            i += 1
        count = min(count, max_repeat)

        result += char * count

    # Final step to reduce repeated characters to at most 3 repetitions
    final_result = ''
    i = 0
    while i < len(result):
        char = result[i]
        count = 0
        while i < len(result) and result[i] == char:
            count += 1
            i += 1
        count = min(count, 3)
        final_result += char * count

    return final_result
copy_df['clean_documents'] = df['reply_text'].apply(modify_string)
copy_df
"""

"\ndef modify_string(s):\n    if not isinstance(s, str):\n        return s  # return the original value if it is not a string\n\n    vowels = 'ะัาำิีึ'\n    max_repeats = {'5': 3, 'ถ': 3}\n    result = ''\n\n    i = 0\n    while i < len(s):\n        char = s[i]\n        max_repeat = max_repeats.get(char, 1 if char in vowels else 1)\n\n        count = 0\n        while i < len(s) and s[i] == char:\n            count += 1\n            i += 1\n        count = min(count, max_repeat)\n\n        result += char * count\n\n    # Final step to reduce repeated characters to at most 3 repetitions\n    final_result = ''\n    i = 0\n    while i < len(result):\n        char = result[i]\n        count = 0\n        while i < len(result) and result[i] == char:\n            count += 1\n            i += 1\n        count = min(count, 3)\n        final_result += char * count\n\n    return final_result\ncopy_df['clean_documents'] = df['reply_text'].apply(modify_string)\ncopy_df\n"

In [ ]:
copy_df['clean_documents'] = copy_df['reply_text'].apply(transform_string)

In [ ]:
df_clean_re=copy_df.copy()

We have a problem with runtime in google colab, so we have to use other software to proceed the function and upload csv file to colab to other processing

In [ ]:
"""
CSV_without_repeating_path='/content/drive/MyDrive/K-DAI/Y2S1/Text mining/Week4/df_without_repeating_alphatbet_vowel_number.csv'
df_clean_re=pd.read_csv(CSV_without_repeating_path)
"""

"\nCSV_without_repeating_path='/content/drive/MyDrive/K-DAI/Y2S1/Text mining/Week4/df_without_repeating_alphatbet_vowel_number.csv'\ndf_clean_re=pd.read_csv(CSV_without_repeating_path)\n"

In [ ]:
df_clean_re

,reply_no,reply_text,clean_documents
6268606,402,พอโดนตบเกรียนแตกเมื่อคืนเลยอาละวาดใหญ่โง่ระดับ...,พอโดนตบเกรียนแตกเมื่อคืนเลยอาละวาดใหญ่โง่ระดับ...
748309,197,ใครมีดอคบ้างวะแบ่งหน่อยปิดรับไปยัง,ใครมีดอคบ้างวะแบ่งหน่อยปิดรับไปยัง
3581181,756,753754เรื่องหลิงซีตอนเช้ากูตัดใจสั่งในชอปปี้ไป...,753754เรื่องหลิงซีตอนเช้ากูตัดใจสั่งในชอปปี้ไป...
7339110,961,นนนนาาาาาาาาาาาาาาาาาาาาาาาาาาาาาาาาาาาาาาาาาา...,นา
7282645,930,929มึงมันโม่งปลอมตัวเป็นแม่ลมแม่ลมตัวจริงจะรับ...,929มึงมันโม่งปลอมตัวเป็นแม่ลมแม่ลมตัวจริงจะรับ...
...,...,...,...
6815187,509,507นึกถึงชื่อริคแฟลร์เลยแหะตอนแรกได้ยินชื่อจาก...,507นึกถึงชื่อริคแฟลร์เลยแหะตอนแรกได้ยินชื่อจาก...
7069447,785,783ยิ่งปลุกพลังได้เอสยิ่งหมาเป็นถึงลูกชายราชาโ...,783ยิ่งปลุกพลังได้เอสยิ่งหมาเป็นถึงลูกชายราชาโ...
5991066,4,3ควย,3ควย
7051003,331,348ในเมื่อมีเศษเกลือให้ใช้จะเก็บไว้ให้บูดทำไมค...,348ในเมื่อมีเศษเกลือให้ใช้จะเก็บไว้ให้บูดทำไมค...


#Update custom word to custom word list

In [ ]:
with open(CUSTOM_WORDS, encoding='utf8') as f:
    words = [line.strip() for line in f]

words


['เอส',
 'เออร์วิน',
 'ชูสุเกะ',
 'ซายะ',
 'อาร์คติค',
 'คามุย',
 'อันเดด',
 'ทากาสุงิ',
 'มาม่า',
 'ไซเบอร์นิวไทป์',
 'โย',
 'หลิงซี',
 'เมมเบอร์',
 'เซนเตอร์',
 'ซิงเกิ้ล',
 'เทลม่อน',
 'ฟานีม่อน',
 'โอฟานีม่อน',
 'โฮลี่ดราม่อน',
 'โฮลี่ริง',
 'ดิจิม่อนไคเซอร์',
 'ดิจิม่อน',
 'ทากาสุงิ',
 'กินทามะ',
 'ฮูเถา',
 'เฉินหลง',
 'ชอปปี้',
 'ทวิตเตอร์',
 'โทนี่',
 'โอฮาโย',
 'แปป',
 'วาคาบะ',
 'คาบุรากิ',
 'วาคาบะ',
 'คิโชวอิน',
 'แม่ง',
 'ไม่มี',
 'นางมาร',
 'หนังสือ',
 'เลเวล',
 'คอมมิค',
 'พิธีจบการศึกษา',
 'ซุยรัน',
 'ลาส',
 'ศีลธรรม',
 'อิงฟ้า',
 'รีแค๊ป',
 'ตอน',
 '555',
 'ถถถ',
 'ซุยรัน',
 'คาบุรากิ',
 'วาคาเบะ',
 'ทาคามิจิ',
 'คาบุรากิมาซายะ',
 'วาคาบะ',
 'คาบุรากิ',
 'วาคาบะจัง',
 'วาคาบะคาบุรากิมาซายะ',
 'จูเลียต',
 'คาบุรากิ',
 'วาคาบะจัง',
 'คาบุรากิ',
 'เรย์กะ',
 'คิมิดอล',
 'เทพ',
 'โทมะ',
 'คุณคิโชวอิน',
 'อุดรธานี',
 'เชียงใหม่',
 'จีน',
 'โอชิ',
 'จงหยวน',
 'ฮูเต๋า']

In [ ]:
custom_words_list = set(thai_words())
print('No of preset words:',len(custom_words_list))
#เพิ่ม customword ของเราลงใน custom_word_list
custom_words_list.update(words)

custom_words_list.remove('ศรี')
custom_words_list.remove('นรี')
print('No of words after update: ',len(custom_words_list))

No of preset words: 62068
No of words after update:  62118


#Apply custom word to tokenizer

In [ ]:

#ตัดคำ

custom_tokenizer = Tokenizer(custom_words_list,engine='newmm')
df_clean_re['clean_documents'] = df_clean_re['reply_text'].apply(lambda x: custom_tokenizer.word_tokenize(x))


In [ ]:
df_clean_re.reset_index(inplace=True)


In [ ]:
df_clean_re

,index,reply_no,reply_text,clean_documents
0,6268606,402,พอโดนตบเกรียนแตกเมื่อคืนเลยอาละวาดใหญ่โง่ระดับ...,"[พอ, โดน, ตบ, เกรียน, แตก, เมื่อคืน, เลย, อาละ..."
1,748309,197,ใครมีดอคบ้างวะแบ่งหน่อยปิดรับไปยัง,"[ใคร, มีด, อค, บ้าง, วะ, แบ่ง, หน่อย, ปิด, รับ..."
2,3581181,756,753754เรื่องหลิงซีตอนเช้ากูตัดใจสั่งในชอปปี้ไป...,"[753754, เรื่อง, หลิงซี, ตอนเช้า, กู, ตัดใจ, ส..."
3,7339110,961,นนนนาาาาาาาาาาาาาาาาาาาาาาาาาาาาาาาาาาาาาาาาาา...,"[นนน, นา, าาาาาาาาาาาาาาาาาาาาาาาาาาาาาาาาาาาา..."
4,7282645,930,929มึงมันโม่งปลอมตัวเป็นแม่ลมแม่ลมตัวจริงจะรับ...,"[929, มึง, มัน, โม่ง, ปลอมตัว, เป็น, แม่, ลม, ..."
...,...,...,...,...
195,6815187,509,507นึกถึงชื่อริคแฟลร์เลยแหะตอนแรกได้ยินชื่อจาก...,"[507, นึกถึง, ชื่อ, ริ, คแฟลร์, เลย, แหะ, ตอนแ..."
196,7069447,785,783ยิ่งปลุกพลังได้เอสยิ่งหมาเป็นถึงลูกชายราชาโ...,"[783, ยิ่ง, ปลุก, พลัง, ได้, เอส, ยิ่ง, หมา, เ..."
197,5991066,4,3ควย,"[3, ควย]"
198,7051003,331,348ในเมื่อมีเศษเกลือให้ใช้จะเก็บไว้ให้บูดทำไมค...,"[348, ใน, เมื่อ, มี, เศษ, เกลือ, ให้, ใช้, จะ,..."


#Prepare clean_documents column for using later

In [ ]:
detokenized_doc = []
for i in range(len(df_clean_re)):
    t = ' '.join(map(str, df_clean_re['clean_documents'][i]))
    detokenized_doc.append(t)
df_clean_re['clean_documents'] = detokenized_doc

In [ ]:
df_clean_re

,index,reply_no,reply_text,clean_documents
0,6268606,402,พอโดนตบเกรียนแตกเมื่อคืนเลยอาละวาดใหญ่โง่ระดับ...,พอ โดน ตบ เกรียน แตก เมื่อคืน เลย อาละวาด ใหญ่...
1,748309,197,ใครมีดอคบ้างวะแบ่งหน่อยปิดรับไปยัง,ใคร มีด อค บ้าง วะ แบ่ง หน่อย ปิด รับ ไป ยัง
2,3581181,756,753754เรื่องหลิงซีตอนเช้ากูตัดใจสั่งในชอปปี้ไป...,753754 เรื่อง หลิงซี ตอนเช้า กู ตัดใจ สั่ง ใน ...
3,7339110,961,นนนนาาาาาาาาาาาาาาาาาาาาาาาาาาาาาาาาาาาาาาาาาา...,นนน นา าาาาาาาาาาาาาาาาาาาาาาาาาาาาาาาาาาาาาาา...
4,7282645,930,929มึงมันโม่งปลอมตัวเป็นแม่ลมแม่ลมตัวจริงจะรับ...,929 มึง มัน โม่ง ปลอมตัว เป็น แม่ ลม แม่ ลม ตั...
...,...,...,...,...
195,6815187,509,507นึกถึงชื่อริคแฟลร์เลยแหะตอนแรกได้ยินชื่อจาก...,507 นึกถึง ชื่อ ริ คแฟลร์ เลย แหะ ตอนแรก ได้ยิ...
196,7069447,785,783ยิ่งปลุกพลังได้เอสยิ่งหมาเป็นถึงลูกชายราชาโ...,783 ยิ่ง ปลุก พลัง ได้ เอส ยิ่ง หมา เป็น ถึง ล...
197,5991066,4,3ควย,3 ควย
198,7051003,331,348ในเมื่อมีเศษเกลือให้ใช้จะเก็บไว้ให้บูดทำไมค...,348 ใน เมื่อ มี เศษ เกลือ ให้ ใช้ จะ เก็บ ไว้ ...


In [ ]:
#Example of using custom word -> Update custom word 'อิงฟ้า' to custom word list
print('Sentence before tokenize:',df_clean_re['reply_text'][46])
print('Tokenize: ',df_clean_re['clean_documents'][46])

Sentence before tokenize: 400อิงฟ้าลาสไอดอล
Tokenize:  400 อิงฟ้า ลาส ไอดอล


In [ ]:
# ตัดคำโดยใช้ช่องว่าง
# tokenization
tokenized_doc = df_clean_re['clean_documents'].fillna('').apply(lambda x: x.split())
tokenized_doc

0      [พอ, โดน, ตบ, เกรียน, แตก, เมื่อคืน, เลย, อาละ...
1      [ใคร, มีด, อค, บ้าง, วะ, แบ่ง, หน่อย, ปิด, รับ...
2      [753754, เรื่อง, หลิงซี, ตอนเช้า, กู, ตัดใจ, ส...
3      [นนน, นา, าาาาาาาาาาาาาาาาาาาาาาาาาาาาาาาาาาาา...
4      [929, มึง, มัน, โม่ง, ปลอมตัว, เป็น, แม่, ลม, ...
                             ...                        
195    [507, นึกถึง, ชื่อ, ริ, คแฟลร์, เลย, แหะ, ตอนแ...
196    [783, ยิ่ง, ปลุก, พลัง, ได้, เอส, ยิ่ง, หมา, เ...
197                                             [3, ควย]
198    [348, ใน, เมื่อ, มี, เศษ, เกลือ, ให้, ใช้, จะ,...
199                                                [756]
Name: clean_documents, Length: 200, dtype: object

#Stop word

1. ใช้ Stop word ที่มีมาให้จาก pythainlp

In [ ]:
'''
# ตรวจสอบ Stop word ที่ Load มา
stopwords = thai_stopwords()
len(stopwords)
'''

'\n# ตรวจสอบ Stop word ที่ Load มา\nstopwords = thai_stopwords()\nlen(stopwords)\n'

In [ ]:
'''stopwords'''

'stopwords'

In [ ]:
'''
new_stopwords=frozenset(new_stopwords_list)
print(len(new_stopwords))
print(len(stopwords))
new_stopwords_list2=stopwords.union(new_stopwords)
print(len(stopwords))
stopwords=new_stopwords_list2
'''


'\nnew_stopwords=frozenset(new_stopwords_list)\nprint(len(new_stopwords))\nprint(len(stopwords))\nnew_stopwords_list2=stopwords.union(new_stopwords)\nprint(len(stopwords))\nstopwords=new_stopwords_list2\n'

2. ใช้ Stop word ที่เราเลือกเอง

In [ ]:
with open(STOP_WORDS) as f:
    new_stopwords_list = [line.strip() for line in f]

new_stopwords_list

['อะ',
 'แต่',
 'ๆ',
 'แม้',
 'กว่า',
 'ได้แก่',
 'เนื่อง',
 'เนื่องด้วย',
 'ถ้า',
 'ถึง',
 'แม้',
 'ถึงแม้',
 'จนกว่า',
 'จึง',
 'ก็',
 'เนื่อง',
 'เนื่องจาก',
 'ว่า',
 'เช่น',
 'เพราะ',
 'เพราะว่า',
 'และ',
 'ก็ตาม',
 'ก็ดี',
 'การ',
 'ความ',
 'วะ',
 'แม่ง',
 'เลย',
 'พอ',
 'โดน',
 'นี่',
 'โน่น',
 'นั่น',
 'นู่น',
 'อีก',
 'นะ',
 'ที่',
 'ที่พึ่ง',
 'บ้าง',
 '']

In [ ]:
stopwords = new_stopwords_list
len(stopwords)
stopwords

['อะ',
 'แต่',
 'ๆ',
 'แม้',
 'กว่า',
 'ได้แก่',
 'เนื่อง',
 'เนื่องด้วย',
 'ถ้า',
 'ถึง',
 'แม้',
 'ถึงแม้',
 'จนกว่า',
 'จึง',
 'ก็',
 'เนื่อง',
 'เนื่องจาก',
 'ว่า',
 'เช่น',
 'เพราะ',
 'เพราะว่า',
 'และ',
 'ก็ตาม',
 'ก็ดี',
 'การ',
 'ความ',
 'วะ',
 'แม่ง',
 'เลย',
 'พอ',
 'โดน',
 'นี่',
 'โน่น',
 'นั่น',
 'นู่น',
 'อีก',
 'นะ',
 'ที่',
 'ที่พึ่ง',
 'บ้าง',
 '']

In [ ]:
tokenized_doc = tokenized_doc.apply(lambda x: [item for item in x if item not in stopwords])
#tokenized_doc = tokenized_doc.apply(lambda x: [item for item in x if len(item) > 2])  <<<<< More Option for delete word that less than 2 alphabet

In [ ]:
#คำว่า: ที่พึ่ง, ที่, วะ, กว่า,
print('Sentence before correcting:',df_clean_re['reply_text'][17])
print('Correcting: ',df_clean_re['clean_documents'][17])

Sentence before correcting: แต่กุไม่เข้าใจหมีจริงๆนะดูก็รู้ว่าไชลด์ที่ไม่ใช่จัดจ้านรีรันมามาเป็ยรอบที่3กับฮูเถาที่พึ่งมารอบ2เป็นระดับ0ทำไมมันเอาโทมะไปโยนลงตู้หลังวะดูยังไงยอดเต้าหู้ก็น่าจะเยอะกว่ามันก็ควรเอาอะไรให้คนมันอยากมากดตู้ไชลด์บ้างดิวะ
Correcting:  แต่ กุ ไม่ เข้าใจ หมี จริงๆ นะ ดู ก็ รู้ ว่า ไช ลด์ ที่ ไม่ ใช่ จัดจ้าน รีรัน มา มา เป็ย รอบ ที่ 3 กับ ฮูเถา ที่พึ่ง มา รอบ 2 เป็น ระดับ 0 ทำไม มัน เอา โทมะ ไป โยน ลง ตู้ หลัง วะ ดู ยังไง ยอด เต้าหู้ ก็ น่าจะ เยอะ กว่า มัน ก็ ควร เอา อะไร ให้ คน มัน อยาก มา กด ตู้ ไช ลด์ บ้าง ดิ วะ


#Correcting the word

In [ ]:
# อ่าน คำผิด และคำที่ถูกต้อง จากไฟล์

with open(CORRECTING_WORDS) as f:
    thai_correct_word_list = [line.strip() for line in f]

thai_correct_word_list

['ครัช ครับ',
 'หรา หรอ',
 'แว็ก แว๊กซ์',
 'เปง เป็น',
 'ฉฉัน ฉัน',
 'ทวิต ทวิตเตอร์',
 'กจน จน',
 'คร้าา ค่า',
 'คร้าาา ค่า',
 'คร่า ค่า',
 'คร่าา ค่า',
 'ก๊อบ ก๊อป',
 'ทำใม ทำไม',
 'รีแค๊ป รีแคป',
 'คอมมิก คอมมิค',
 'พร๊อท พล๊อต',
 'กระจ๊ก กระจ๊อก',
 'มบ มา',
 'ตอด ตลอด',
 'ศรีธรรม ศีลธรรม',
 'กจน กัน',
 'หน่ออย หน่อย',
 'ซูด สูด',
 'ม่าย ไม่',
 'หวาย ไหว',
 'เป็ย เป็น']

In [ ]:
# ดึงคู่คำ คำผิด และคำที่ถูกต้อง ลง Dict
correct_pair={}

for pair in thai_correct_word_list:
    correct_pair[pair.split()[0].strip()]=pair.split()[1].strip()

In [ ]:
# แก้คำผิด
doc = []
for tokenized_list in tokenized_doc:
    for index, word in enumerate(tokenized_list):
        if word in correct_pair:
            tokenized_list[index]=correct_pair[word]
    doc.append(tokenized_list)

tokenized_doc = pd.Series(doc)

In [ ]:
# de-tokenization
detokenized_doc = []
for i in range(len(copy_df)):
    t = ' '.join(tokenized_doc[i])
    detokenized_doc.append(t)
df_clean_re['clean_documents'] = detokenized_doc

In [ ]:
df_clean_re

,index,reply_no,reply_text,clean_documents
0,6268606,402,พอโดนตบเกรียนแตกเมื่อคืนเลยอาละวาดใหญ่โง่ระดับ...,ตบ เกรียน แตก เมื่อคืน อาละวาด ใหญ่ โง่ ระดับ ...
1,748309,197,ใครมีดอคบ้างวะแบ่งหน่อยปิดรับไปยัง,ใคร มีด อค แบ่ง หน่อย ปิด รับ ไป ยัง
2,3581181,756,753754เรื่องหลิงซีตอนเช้ากูตัดใจสั่งในชอปปี้ไป...,753754 เรื่อง หลิงซี ตอนเช้า กู ตัดใจ สั่ง ใน ...
3,7339110,961,นนนนาาาาาาาาาาาาาาาาาาาาาาาาาาาาาาาาาาาาาาาาาา...,นนน นา าาาาาาาาาาาาาาาาาาาาาาาาาาาาาาาาาาาาาาา...
4,7282645,930,929มึงมันโม่งปลอมตัวเป็นแม่ลมแม่ลมตัวจริงจะรับ...,929 มึง มัน โม่ง ปลอมตัว เป็น แม่ ลม แม่ ลม ตั...
...,...,...,...,...
195,6815187,509,507นึกถึงชื่อริคแฟลร์เลยแหะตอนแรกได้ยินชื่อจาก...,507 นึกถึง ชื่อ ริ คแฟลร์ แหะ ตอนแรก ได้ยิน ชื...
196,7069447,785,783ยิ่งปลุกพลังได้เอสยิ่งหมาเป็นถึงลูกชายราชาโ...,783 ยิ่ง ปลุก พลัง ได้ เอส ยิ่ง หมา เป็น ลูกชา...
197,5991066,4,3ควย,3 ควย
198,7051003,331,348ในเมื่อมีเศษเกลือให้ใช้จะเก็บไว้ให้บูดทำไมค...,348 ใน เมื่อ มี เศษ เกลือ ให้ ใช้ จะ เก็บ ไว้ ...


In [ ]:
#Example of using custom word -> Update custom word 'อิงฟ้า' to custom word list
print('Sentence before correcting:',df_clean_re['reply_text'][78])
print('Correcting: ',df_clean_re['clean_documents'][78])


Sentence before correcting: เกิน8บรรทัดกูอ่านม่ายหวาย
Correcting:  เกิน 8 บรรทัด กู อ่าน ไม่ ไหว


#Put the clean documents to W2V

In [ ]:
df_clean_re['clean_documents'][0:5]

0    ตบ เกรียน แตก เมื่อคืน อาละวาด ใหญ่ โง่ ระดับ ...
1                 ใคร มีด อค แบ่ง หน่อย ปิด รับ ไป ยัง
2    753754 เรื่อง หลิงซี ตอนเช้า กู ตัดใจ สั่ง ใน ...
3    นนน นา าาาาาาาาาาาาาาาาาาาาาาาาาาาาาาาาาาาาาาา...
4    929 มึง มัน โม่ง ปลอมตัว เป็น แม่ ลม แม่ ลม ตั...
Name: clean_documents, dtype: object

In [ ]:
sentences = df_clean_re['clean_documents'].values
sentences.shape

(200,)

In [ ]:
sentences[0:5]

array(['ตบ เกรียน แตก เมื่อคืน อาละวาด ใหญ่ โง่ ระดับ ตู่ มึง เนี่ย ถถถ ถ',
       'ใคร มีด อค แบ่ง หน่อย ปิด รับ ไป ยัง',
       '753754 เรื่อง หลิงซี ตอนเช้า กู ตัดใจ สั่ง ใน ชอปปี้ ไป ทั้งที่ กะ จะ รอ ไป เอา งานหนังสือ กู กลัว หมด ชิบหาย ใน ทวิ ต เห็น เริ่ม ฮิต กัน แล้ว',
       'นนน นา าาาาาาาาาาาาาาาาาาาาาาาาาาาาาาาาาาาาาาาาาาาาาาาาาาาาาาาาาาาาาา',
       '929 มึง มัน โม่ง ปลอมตัว เป็น แม่ ลม แม่ ลม ตัวจริง จะ รับ ไม่ ได้ที่ ลูกสาว คนสวย มี ดุ้น ยักษ์ แล้วไป เอา โจร นม แน่น เป็น เมีย แน่นอน'],
      dtype=object)

In [ ]:
tsentences = []
for st in sentences:
    tokens = word_tokenize(st.strip())
    tsentences.append(tokens)

tsentences[0:2]



[['ตบ',
  ' ',
  'เกรียน',
  ' ',
  'แตก',
  ' ',
  'เมื่อคืน',
  ' ',
  'อาละวาด',
  ' ',
  'ใหญ่',
  ' ',
  'โง่',
  ' ',
  'ระดับ',
  ' ',
  'ตู่',
  ' ',
  'มึง',
  ' ',
  'เนี่ย',
  ' ',
  'ถถถ',
  ' ',
  'ถ'],
 ['ใคร',
  ' ',
  'มีด',
  ' ',
  'อค',
  ' ',
  'แบ่ง',
  ' ',
  'หน่อย',
  ' ',
  'ปิด',
  ' ',
  'รับ',
  ' ',
  'ไป',
  ' ',
  'ยัง']]

In [ ]:
tsentences = [[word for word in tsentence if word.strip() != ''] for tsentence in tsentences]
tsentences[0:2]

[['ตบ',
  'เกรียน',
  'แตก',
  'เมื่อคืน',
  'อาละวาด',
  'ใหญ่',
  'โง่',
  'ระดับ',
  'ตู่',
  'มึง',
  'เนี่ย',
  'ถถถ',
  'ถ'],
 ['ใคร', 'มีด', 'อค', 'แบ่ง', 'หน่อย', 'ปิด', 'รับ', 'ไป', 'ยัง']]

In [ ]:
dimension = 128
w2v_model = Word2Vec(tsentences, min_count=1, vector_size=dimension, workers=6, sg=1, epochs=50)

In [ ]:
print(w2v_model)

Word2Vec<vocab=2039, vector_size=128, alpha=0.025>


In [ ]:
w2v_model.save('w2v_model.bin')

In [ ]:
new_model = Word2Vec.load('w2v_model.bin')

In [ ]:
#words = list(new_model.wv.vocab)
words = list(new_model.wv.index_to_key)

In [ ]:
words

['ไม่',
 'มา',
 'กู',
 'มี',
 'มัน',
 'ไป',
 'เป็น',
 'ได้',
 'แล้ว',
 'จะ',
 'คน',
 'ให้',
 'มึง',
 'ใน',
 'กับ',
 '555',
 'ของ',
 'ด้วย',
 '1',
 'อะไร',
 'นี้',
 'กัน',
 'คือ',
 'ผม',
 'อยู่',
 'พวก',
 'เรื่อง',
 'คา',
 'เอา',
 'แค่',
 'แบบ',
 'นั้น',
 'เขา',
 'บอก',
 'อย่าง',
 'ดี',
 'ต้อง',
 'ทำไม',
 'ไหน',
 'รา',
 'เห็น',
 'จริงๆ',
 'ยัง',
 'ฉัน',
 'ใช่',
 'หน้า',
 'ดู',
 'มาก',
 'กุ',
 'ใช้',
 'ลง',
 'ใคร',
 'จัง',
 'รู้',
 'ละ',
 'ทำ',
 'ตอน',
 'ไง',
 'ชอบ',
 'ส',
 'เนย',
 'ตัว',
 'วา',
 'ล่ะ',
 'เวล',
 'กะ',
 'รัก',
 'อ่าน',
 'ไอ้',
 'มั้ย',
 'คิด',
 'กิ',
 'เคย',
 'จาก',
 'ก่อน',
 'เด็ก',
 'เยอะ',
 'บะ',
 'น',
 'ค่ะ',
 'อยาก',
 'หน่อย',
 'ทุกคน',
 'วัน',
 'ว่ะ',
 'เล่น',
 'เซ',
 'อ่ะ',
 '2',
 'น้อง',
 'หรือ',
 'ขอ',
 'ย',
 'เขียน',
 'เกม',
 'เทพ',
 'ฮาเร็ม',
 'ออก',
 'จบ',
 'เหมือน',
 'ชื่อ',
 'เรา',
 'ติ่ง',
 'ท่าน',
 'เข้า',
 'ส่ง',
 'ทรู',
 'บอ',
 'แรก',
 '3',
 'รู้สึก',
 'ก็ได้',
 'นาน',
 'ใส่',
 'ไหม',
 'ที',
 'ใหม่',
 'ยังไง',
 'นั่ง',
 'แบบนี้',
 'เรย์',
 'เริ่ม',
 'ไร'

In [ ]:
vectors = list(new_model.wv.vectors)

In [ ]:
vectors

Download w2vG and labelG

In [ ]:
import csv

with open('w2vG2.tsv', 'w') as out_file:
    tsv_writer = csv.writer(out_file, delimiter='\t')
    for v in vectors:
        tsv_writer.writerow(v)

with open('labelG2.tsv', 'w') as out_file:
    tsv_writer = csv.writer(out_file, delimiter='\t')
    for word in words:
        tsv_writer.writerow([word])